https://docs.sqlalchemy.org/en/13/orm/tutorial.html

In [1]:
import sqlite3

In [2]:
conn = sqlite3.connect('data/sqlite3/test.db')
conn.execute('DROP TABLE IF EXISTS user_account')
conn.commit()

In [3]:
from typing import List
from typing import Optional
from sqlalchemy import ForeignKey
from sqlalchemy import String
from sqlalchemy.orm import DeclarativeBase
from sqlalchemy.orm import Mapped
from sqlalchemy.orm import mapped_column
from sqlalchemy.orm import relationship
from sqlalchemy import create_engine

In [4]:
path = ! pwd

# windows
# path = ! cd
engine = create_engine(f'sqlite:///{path[0]}/data/sqlite3/test.db')

# Defino modelo

In [5]:
class Base(DeclarativeBase):
    pass

class User(Base):
    __tablename__ = "user_account"
    id: Mapped[int] = mapped_column(primary_key=True)
    name: Mapped[str] = mapped_column(String(30))
    fullname: Mapped[Optional[str]]
    age: Mapped[Optional[int]]
    addresses: Mapped[List["Address"]] = relationship(
        back_populates="user", cascade="all, delete-orphan"
    )
    
    def __repr__(self) -> str:
        return f"User(id={self.id!r}, name={self.name!r}, fullname={self.fullname!r})"

class Address(Base):
    __tablename__ = "address"
    id: Mapped[int] = mapped_column(primary_key=True)
    email_address: Mapped[str]
    user_id: Mapped[int] = mapped_column(ForeignKey("user_account.id"))
    user: Mapped["User"] = relationship(back_populates="addresses")

    def __repr__(self) -> str:
        return f"Address(id={self.id!r}, email_address={self.email_address!r})"

# Creo la tabla

In [6]:
# nos crea automaticamente el Schema
User.__table__ 

Table('user_account', MetaData(), Column('id', Integer(), table=<user_account>, primary_key=True, nullable=False), Column('name', String(length=30), table=<user_account>, nullable=False), Column('fullname', String(), table=<user_account>), Column('age', Integer(), table=<user_account>), schema=None)

In [7]:
Base.metadata.create_all(engine)

In [8]:
pedro = User(
    name="Pedro",
    fullname="Pedro Gomez",
    age=45,
    addresses=[Address(email_address="pedro@sqlalchemy.org")],
)

In [9]:
pedro.name, pedro.fullname, pedro.addresses, pedro.age

('Pedro',
 'Pedro Gomez',
 [Address(id=None, email_address='pedro@sqlalchemy.org')],
 45)

# Creación de la sessión

In [10]:
from sqlalchemy.orm import Session
session = Session(engine)

# Agregamos usuarios

In [11]:
ed_user = User(name='ed', fullname='Ed Jones', age=45)
session.add(ed_user)
session.commit()
session.close()

In [12]:
with Session(engine) as session:
    spongebob = User(
        name="spongebob",
        fullname="Spongebob Squarepants",
        age=40,
        addresses=[Address(email_address="spongebob@sqlalchemy.org")],
    )
    sandy = User(
        name="sandy",
        fullname="Sandy Cheeks",
        age=40,
        addresses=[
            Address(email_address="sandy@sqlalchemy.org"),
            Address(email_address="sandy@squirrelpower.org"),
        ],
    )
    patrick = User(name="patrick", fullname="Patrick Star")
    session.add_all([spongebob, sandy, patrick])
    session.commit()

# Query

In [13]:
from sqlalchemy import select

session = Session(engine)

stmt = select(User).where(User.name.in_(["spongebob", "sandy"]))

for user in session.scalars(stmt):
    print(user)

User(id=2, name='spongebob', fullname='Spongebob Squarepants')
User(id=3, name='sandy', fullname='Sandy Cheeks')


In [14]:
session.query(User).count()

4

# Agregamos mas usuarios

In [15]:
# Agregamos otros usuarios
session.add_all([User(name='wendy', fullname='Wendy Williams', age=35),
                 User(name='mary', fullname='Mary Contrary', age=35),
                 User(name='john', fullname='John Lennon', age=20),
                 User(name='fred', fullname='Fred Flintstone', age=20)])

In [16]:
session.query(User).count()

8

# Session new and dirty

In [17]:
juan_user = User(name='juan', fullname='Juan Perez')
session.add(juan_user)

In [18]:
session.new

IdentitySet([User(id=None, name='juan', fullname='Juan Perez')])

In [19]:
session.dirty

IdentitySet([])

In [20]:
session.commit()

In [21]:
session.new

IdentitySet([])

In [22]:
juan_user.name = 'Juan'
session.add(juan_user)

In [23]:
session.dirty

IdentitySet([User(id=9, name='Juan', fullname='Juan Perez')])

In [24]:
session.commit()

In [25]:
session.dirty

IdentitySet([])

# Volviendo atras (Rollback)

In [26]:
juan_user.name = 'John'

In [27]:
session.dirty

IdentitySet([User(id=9, name='John', fullname='Juan Perez')])

In [28]:
fake_user = User(name='fakeuser', fullname='Invalid')
session.add(fake_user)

In [29]:
session.new

IdentitySet([User(id=None, name='fakeuser', fullname='Invalid')])

In [30]:
session.rollback()

In [31]:
session.dirty

IdentitySet([])

In [32]:
session.new

IdentitySet([])

# Ordenando

In [34]:
session.query(User).order_by(User.age).all()

[User(id=4, name='patrick', fullname='Patrick Star'),
 User(id=9, name='Juan', fullname='Juan Perez'),
 User(id=7, name='john', fullname='John Lennon'),
 User(id=8, name='fred', fullname='Fred Flintstone'),
 User(id=5, name='wendy', fullname='Wendy Williams'),
 User(id=6, name='mary', fullname='Mary Contrary'),
 User(id=2, name='spongebob', fullname='Spongebob Squarepants'),
 User(id=3, name='sandy', fullname='Sandy Cheeks'),
 User(id=1, name='ed', fullname='Ed Jones')]

In [36]:
for instance in session.query(User).order_by(User.age):
    print(instance.id, instance.name, instance.fullname, instance.age)

4 patrick Patrick Star None
9 Juan Juan Perez None
7 john John Lennon 20
8 fred Fred Flintstone 20
5 wendy Wendy Williams 35
6 mary Mary Contrary 35
2 spongebob Spongebob Squarepants 40
3 sandy Sandy Cheeks 40
1 ed Ed Jones 45


# Agregación

In [37]:
import sqlalchemy

session.query(User.age, sqlalchemy.func.count(User.age)).group_by(User.age).all()

[(None, 0), (20, 2), (35, 2), (40, 2), (45, 1)]

In [38]:
session.query(sqlalchemy.func.avg(User.age)).all()

[(33.57142857142857,)]

# Filtros

In [39]:
session.query(User).filter(User.age == 20).all()

[User(id=7, name='john', fullname='John Lennon'),
 User(id=8, name='fred', fullname='Fred Flintstone')]